In [1]:
import torch
from schnetpack.datasets import QM9
import schnetpack as spk
import os
from my_config import config_args
from Model.HGDM import HyperbolicAE,HyperbolicDiffusion
import optimizers
import numpy as np
import logging
import time
from tqdm import tqdm
from Model import Encoders, Decoders

In [2]:
qm9data = QM9('./data/qm9.db', download=True,load_only=[QM9.U0])
qm9split = './data/qm9split'
print(len(qm9data))

train, val, test = spk.train_test_split(
        data=qm9data,
        num_train=30000,
        num_val=10000,
        split_file=os.path.join(qm9split, "split30000-10000.npz"),
    )
print(len(train),len(val),len(test))

train_loader = spk.AtomsLoader(train, batch_size=256, shuffle=False)
val_loader = spk.AtomsLoader(val, batch_size=256)

133885
30000 10000 93885


In [3]:
import json
class obj(object):
    def __init__(self, dict_):
        self.__dict__.update(dict_)
args = json.loads(json.dumps(config_args), object_hook=obj)

model = HyperbolicAE(args)

optimizer = getattr(optimizers, args.optimizer)(params=model.parameters(), lr=args.lr,
                                                    weight_decay=args.weight_decay)
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=args.lr_reduce_freq,
    gamma=float(args.gamma)
)
tot_params = sum([np.prod(p.size()) for p in model.parameters()])
print(f"Total number of parameters: {tot_params}")
device = torch.device('cuda')
# Train model
t_total = time.time()

Total number of parameters: 7112


In [4]:
encoder_path = './saved_model/HNN-encoder.pt'
decoder_path = './saved_model/HNN-decoder.pt'

encoder = getattr(Encoders, args.model)(args)
encoder.load_state_dict(torch.load(encoder_path))
decoder = Decoders.model2decoder[args.model](encoder.curvatures, args)
decoder.load_state_dict(torch.load(decoder_path))

<All keys matched successfully>

In [5]:
model = HyperbolicDiffusion(args,encoder,decoder)
model = model.to(device)
model.eval()
with torch.no_grad():
    loss_sum,n = 0.0,0
    for input in tqdm(train_loader):
        for key in input:
            input[key] = input[key].to(torch.device('cuda'))
        t = time.time()
        model.train()
        optimizer.zero_grad()
        loss = model(input)
        print(loss)
        n += 1
        loss_sum += loss

Gamma schedule:
[-5.         -4.693569   -4.3871384  -4.0810676  -3.7749963  -3.4689255
 -3.1628542  -2.8564234  -2.5499928  -2.2439218  -1.9378507  -1.6317797
 -1.3253489  -1.0192778  -0.71320677 -0.4074955  -0.10178375  0.20500612
  0.5110779   0.8171487   1.12322     1.4296503   1.7353616   2.0421524
  2.3482232   2.654294    2.9603653   3.2664366   3.572507    3.8785782
  4.1846495   4.4907207   4.796792    5.1028633   5.4089346   5.715005
  6.021076    6.3271475   6.6328583   6.9389296   7.24572     7.5514307
  7.857502    8.163933    8.469285    8.775716    9.081428    9.387858
  9.693929   10.        ]


  0%|          | 0/118 [00:01<?, ?it/s]

t: torch.Size([256, 1])


ValueError: only one element tensors can be converted to Python scalars